In [ ]:
# Background Links

This code should set up our sniffer! Based on:
* http://hackoftheday.securitytube.net/2013/03/wi-fi-sniffer-in-10-lines-of-python.html


Other links that could be useful:
* http://lifehacker.com/access-os-xs-packet-sniffer-with-the-wireless-diagnosti-1662764826
* https://supportforums.cisco.com/document/75221/wireless-sniffing-using-mac-os-x-106-and-above
* http://jacobsalmela.com/os-x-yosemite-secret-packet-sniffer/
* http://osxdaily.com/2015/04/23/sniff-packet-capture-packet-trace-mac-os-x-wireless-diagnostics/
* https://diogomonica.com/sniffing-in-monitor-mode-with-airport/
* http://www.cqure.net/wp/2014/04/scapy-with-wifi-monitor-rfmon-mode-on-os-x/
* https://theitgeekchronicles.files.wordpress.com/2012/05/scapyguide1.pdf
* https://github.com/brannondorsey/ProbeKit/wiki/Batch-Downloading-Wigle.net-Data

And for the details of an 802.11 frame:
* https://supportforums.cisco.com/document/52391/80211-frames-starter-guide-learn-wireless-sniffer-traces

A paper on doing tracing:
* http://www.hindawi.com/journals/ijdsn/2013/927940/

# Scapy Setup

In [ ]:
from scapy.all import *

# Load Manufacturers MAC List

Got the manufactureers list from: https://code.wireshark.org/review/gitweb?p=wireshark.git;a=blob_plain;f=manuf

In [ ]:
MAC_manufacturers = dict()
for line in open("manufOriginal.txt"):
    words1 = line.split("\t")
    if(len(words1) > 1):
        words2 = words1[1].split("#")
        MAC_manufacturers[words1[0]] = words2[0].strip()

In [23]:
print(len(MAC_manufacturers))
MAC_manufacturers["60:A3:7D"]

NameError: name 'MAC_manufacturers' is not defined

# Dynamic Monitoring Version

Import the scapy package that dos this sniffing.

Set up the fuinction that isolates probe packets

In [ ]:
ap_list = []

def PacketHandler(pkt) :

    if pkt.haslayer(Dot11) :
        if pkt.type == 0 and pkt.subtype == 8 :
            if pkt.addr2 not in ap_list :
                ap_list.append(pkt.addr2)
                print "AP MAC: %s with SSID: %s " %(pkt.addr2, pkt.info)

Call the sniff function to start sniffing

In [ ]:
sniff(iface="en0", prn = PacketHandler, count=400)

# Static File Version

A version that opens a wcap file and parses the packets in it.

In [ ]:
# Read a list of packets saved off by wireless diagnostics
pkts = rdpcap("/Users/bmacnamee/Desktop/2016.02.26_15-05-54-GMT.wcap")

In [ ]:
import os
for file in os.listdir("/Users/bmacnamee/Desktop/"):
    if file.endswith(".txt"):
        print(file)

Find all of the .wcap files on the desktop and open the most recent one.

In [ ]:
import glob, os
os.chdir("/Users/bmacnamee/Desktop/")
wcapFiles = glob.glob("*.wcap")
wcapFiles.sort()
wcapFiles.reverse()
pkts = rdpcap(wcapFiles[0])

In [ ]:
# Print summaries of the packets saved off
#print(pkts.nsummary())
print(pkts[0].summary())
print(pkts[0].info)
print(type(pkts[0]))
#help(scapy.layers.dot11.RadioTap)
pkts[0].show()

In [ ]:
devices = dict()

counter = 0

# Loop through the packets in the file and take a look at them
for pkt in pkts:

    if pkt.haslayer(Dot11) :
        # Type = 0 is a management frame
        # Subtype 8 is a beacon frame
        # Subtype 4 is a probe request frame
        # Subtype 5 is a probe response frame
        if pkt.type == 0 and pkt.subtype == 4 :

            counter += 1
           
            if not pkt.addr2 in devices:
                
                # Isolate the manufacturers code
                manuCode = pkt.addr2[0:8]
                if manuCode in MAC_manufacturers:
                    manu = MAC_manufacturers[manuCode]
                else:
                    manu = ""
                    
                devices[pkt.addr2] = {"freq":1, "last_seen":pkt.time, "manu":manu, "ssids":set()}

                # If there is an SSID listed (in info attribute) record it
                if hasattr(pkt, 'info'):
                    ssid = pkt.info
                    ssid = ssid.strip()
                    #ssid = ''.join(filter(lambda x:x in string.printable, ssid))
                    
                    # Not very good, but simple error handling - ssids of more than 32 charcters probably indicate malformed packets.
                    #if(len(ssid) <= 32 and ssid.find("\x00") == -1):
                    if(all(c in string.printable for c in ssid) and (len(ssid) > 5)):
                        devices[pkt.addr2]["ssids"].add(ssid)

            else:
                # Increment appearance frequency
                devices[pkt.addr2]["freq"] += 1

                # If this occurrence is more recent that thae recorded one update last_seen
                if devices[pkt.addr2]["last_seen"] < pkt.time:
                    devices[pkt.addr2]["last_seen"] = pkt.time

                # If there is an SSID listed (in info attribute) record it
                if hasattr(pkt, 'info'):
                    ssid = pkt.info
                    ssid = ssid.strip()
                    #ssid = ''.join(filter(lambda x:x in string.printable, ssid))
                    # Not very good, but simple error handling - ssids of more than 32 charcters probably indicate malformed packets.
                    #if(len(ssid) <= 32 and ssid.find("\x00") == -1):
                    if(all(c in string.printable for c in ssid) and (len(ssid) > 5)):
                        devices[pkt.addr2]["ssids"].add(ssid)
                        
            

In [ ]:
len(devices)
print(devices)

Write out the devices read in into a nice csv format

In [ ]:
import csv
import time
with open('devices' +  str(time.time()) + '.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['ID', 'freq', 'manu', 'ssids', 'last_seen'])
    
    for d in devices:
        ssidString = " | ".join(str(e) for e in devices[d]['ssids'])
        writer.writerow([d, devices[d]['freq'] , devices[d]['manu'], ssidString, devices[d]['last_seen']])
                        

Generate a list of ssids found

In [ ]:
ssidsFound = []

# Iterate through the devices list
for d in devices:
    
    # get the list of ssids saved from a device
    ssids = devices[d]["ssids"]
    
    # loop through the SSIDs
    for ssid in ssids:

        ssidsFound.append(ssid)

ssidsFound = list(set(ssidsFound))
print(len(ssidsFound))
print(ssidsFound)

# Generate HTML Page

<!DOCTYPE html>
<html lang="en">
<head>
  <title>Bootstrap Example</title>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.0/jquery.min.js"></script>
  <script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
</head>
<body>

<div class="container-fluid">
  <div class="row">
    <div class="col-sm-2" style="background-color:lavender;">Device</div>
    <div class="col-sm-2" style="background-color:lavenderblush;">.col-sm-4</div>
    <div class="col-sm-2" style="background-color:lavender;">.col-sm-4</div>
    <div class="col-sm-2" style="background-color:lavender;">.col-sm-4</div>
    <div class="col-sm-2" style="background-color:lavenderblush;">.col-sm-4</div>
    <div class="col-sm-2" style="background-color:lavender;">.col-sm-4</div>
  </div>
</div>

</body>
</html>

In [ ]:
devicesPerRow = 4
numDevicesFound = len(devices)

deviceCount = 0


with open('devicesFound.html', 'w') as htmlfile:
    
    # Write the HTML header
    htmlfile.write("<!DOCTYPE html>\n")
    htmlfile.write("<html lang=\"en\">\n")
    htmlfile.write("<head>\n")
    htmlfile.write("<title>Devices Found</title>\n")
    htmlfile.write("<meta charset=\"utf-8\">\n")
    htmlfile.write("<meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">\n")
    htmlfile.write("<link rel=\"stylesheet\" href=\"http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css\">\n")
    htmlfile.write("<script src=\"https://ajax.googleapis.com/ajax/libs/jquery/1.12.0/jquery.min.js\"></script>\n")
    htmlfile.write("<script src=\"http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js\"></script>\n")
    htmlfile.write("<style type=\"text/css\"> h1.panel-title {color:#0033cc; font-size:24px}  </style>")
        
    htmlfile.write("</head>\n")
    htmlfile.write("<body>\n")
    
    # Start the devices block
    htmlfile.write("<div class=\"container-fluid\">\n")
    
    # Loop through each device
    #for d in devices:
    for d in sorted(devices, key=devices.get, reverse=True):
        
        # If we need to start a new row start one
        if (deviceCount % devicesPerRow == 0):
            htmlfile.write("\t<div class=\"row\">\n")
        
        # Write our the details of a device
        #htmlfile.write("\t\t<div class=\"col-sm-3\" style=\"border:solid 2px black;background-color:grey;\" >\n")
        htmlfile.write("\t\t<div class=\"col-sm-3\">\n")
        htmlfile.write("<div class = \"panel panel-default\">\n")
        htmlfile.write("<div class = \"panel-heading\">\n")
        htmlfile.write("<h1 class = \"panel-title\">" + d + "</h1>\n")
        htmlfile.write("</div>\n")
   
        htmlfile.write("<div class = \"panel-body\">\n")
        htmlfile.write("\t\t\t<h3>Handset: " + devices[d]['manu'] + " </h3>\n")
        htmlfile.write("\t\t\t<h3>Strength: " + str(devices[d]['freq']) + "</h3>\n")
        htmlfile.write("\t\t\t<h3>" + str(devices[d]['last_seen']) + "</h3>\n")
        ssidString = ", ".join(str(e) for e in devices[d]['ssids'])
        htmlfile.write("\t\t\t<p>" + ssidString + "</p>\n")
        htmlfile.write("</div>\n")
        htmlfile.write("</div>\n")

        #htmlfile.write("\t\t\t<h1>" + d + "</h1>\n")
        #htmlfile.write("\t\t\t<h2>Handset: " + devices[d]['manu'] + " </h2>\n")
        #htmlfile.write("\t\t\t<h2>Strength: " + str(devices[d]['freq']) + "</h2>\n")
        #htmlfile.write("\t\t\t<h2>" + str(devices[d]['last_seen']) + "</h2>\n")
        #ssidString = ", ".join(str(e) for e in devices[d]['ssids'])
        #htmlfile.write("\t\t\t<p>" + ssidString + "</p>\n")
        htmlfile.write("\t\t</div>\n")
        

        deviceCount += 1
        
        # If we need to end a row end one
        if (deviceCount % devicesPerRow == 0):
            htmlfile.write("\t</div>\n")
    
    # If we didn't just end a row, end one
    if (deviceCount % devicesPerRow != 0):
        htmlfile.write("\t</div>\n")
    
    # End the block with a </div>
    htmlfile.write("</div>\n")
    

    htmlfile.write("</body>\n")
    htmlfile.write("</html>\n")

# Wigle

Find network location from Wigle

In [ ]:
from wigle import *

# connect to wigle
wigle = Wigle("brian.macn", "hardeight")

ssidName = "RDS-FREEWIFI"
#ssidName = "TheHurlers"

# search for an ssid
#rds = wigle.search(ssid="RDS-FREEWIFI")
searchRes = wigle.search(ssid=ssidName)

if (len(searchRes) > 0):
    # sort by last time seen
    newlist = sorted(searchRes, key=lambda k: k['lasttime'])

    # average out lat and long
    ssid_lat = 0
    ssid_long = 0
    for entry in newlist:
        ssid_lat += entry["trilat"]
        ssid_long += entry["trilong"]
    ssid_lat = ssid_lat/len(newlist)
    ssid_long = ssid_long/len(newlist)
    
    print "Found ", ssidName, ssid_lat, ssid_long

else:
    
    print "Can't find", ssidName
        

Define Wigle lookup function

In [ ]:
from wigle import *

def findWigleSSID(name):

    # Define the Ireland lat and long ranges
    irelandLatRange = ((50.9847, 55.5398))
    irelandLongRange = ((-10.7297, -4.495))

    # connect to wigle
    wigle = Wigle("brian.macn", "hardeight")

    ssidName = "RDS-FREEWIFI"
    #ssidName = "TheHurlers"

    # search for an ssid
    searchRes = wigle.search(ssid=ssidName, lat_range=irelandLatRange, long_range=irelandLongRange)

    if (len(searchRes) > 0):
        # sort by last time seen
        newlist = sorted(searchRes, key=lambda k: k['lasttime'])

        # average out lat and long
        ssid_lat = 0
        ssid_long = 0
        for entry in newlist:
            ssid_lat += entry["trilat"]
            ssid_long += entry["trilong"]
        ssid_lat = ssid_lat/len(newlist)
        ssid_long = ssid_long/len(newlist)

        print "Found ", ssidName, ssid_lat, ssid_long

    else:

        print "Can't find", ssidName
        
    return {"ssid":ssidName, "lat":ssid_lat,"long":ssid_long}

Rather than hammer Wigle with seraches per SSID, download all of Ireland!

In [3]:
from wigle import *

# Define the Ireland lat and long ranges
irelandLatRange = ((53.1236, 53.5419))
irelandLongRange = ((-6.9967, -5.4271))

# connect to wigle
wigle = Wigle("brian.macn", "hardeight")

# search for all ssids within Ireland
wigleInIreland = wigle.search(lat_range=irelandLatRange, long_range=irelandLongRange, max_results=1000)
len(wigleInIreland)

WigleAuthenticationError: Could not authenticate as user brian.macn

Write the Wigle results out to a json file

In [ ]:
#help(wigle.search)
len(wigleInIreland)
wigleInIreland

import json
with open('wigleBigDump.txt', 'w') as outfile:
    json.dump(wigleInIreland, outfile)

In [16]:
from pygle import network
network.search.__doc__

"Search the WiGLE database\n    \n    Query the WiGLE database for paginated results based on multiple criteria. \n    API and session authentication default to a page size of 100 results/page. \n    COMMAPI defaults to a page size of 25 with a maximum of 1000 results per \n    return. Number of daily queries allowed per user are throttled based on \n    history and participation\n    \n    Parameters\n    ----------\n    onlymine : boolean, optional\n        Search only for points first discovered by the current user. Use any \n        string to set, leave unset for general search. Can't be used with \n        COMMAPI auth, since these are points you have locally.\n    first : int, optional\n        Result offset to fetch, used to page through results. Defaults to 0. \n        COMMAPI queries are bounded at 100 pages. If you need more open-ended \n        searches than this, we recommend contacting WiGLE-admin@wigle.net to \n        discuss bulk pricing.\n    latrange1 : float, option

In [4]:
from pygle import network
limerickLatLow = 52.547626
limerickLatHigh = 52.752432
limerickLongLow = -8.860613
limerickLongHigh = -8.374656

regionLatLow = limerickLatLow
regionLatHigh = limerickLatHigh
regionLongLow = limerickLongLow
regionLongHigh = limerickLongHigh
dumpFileName = 'limerick'

regionLatRange = (regionLatLow, regionLatHigh)
regionLongRange = (regionLongLow, regionLongHigh)

        
#network.search(lat_range=regionLatRange, long_range=regionLongRange, max_results=1000)
network.search(latrange1=regionLatLow, latrange2=regionLatHigh, longrange1=regionLongLow, longrange2=regionLongHigh, resultsPerPage=1000)


ImportError: No module named pygle

Iterate across Dublin in steps writing out json file each time

In [6]:
from wigle import *
from pygle import network
import json
import time

# Define the Dublin lat and long ranges
dublinLatLow = 53.312996
dublinLatHigh = 53.359327
dublinLongLow = -6.297039
dublinLongHigh = -6.170696

irelandLatLow = 51.004739
irelandLatHigh = 55.409966
irelandLongLow = -10.808161
irelandLongHigh = -5.435289

galwayLatLow = 53.267806
galwayLatHigh = 53.341405
galwayLongLow = -9.022181
galwayLongHigh = -8.904250

derryLatLow = 53.677144 
derryLatHigh = 55.329782
derryLongLow = -8.738483
derryLongHigh = -5.239252

derryCityLatLow = 54.921410
derryCityLatHigh = 55.053389
derryCityLongLow = -7.423315
derryCityLongHigh = -7.179218

corkLatLow = 51.685755
corkLatHigh = 52.125477
corkLongLow = -9.005045
corkLongHigh = -7.870149

limerickLatLow = 51.547626
limerickLatHigh = 53.752432
limerickLongLow = -9.860613
limerickLongHigh = -7.374656

birminghamLatLow = 52.3912092
birminghamLatHigh = 52.48012
birminghamLongLow = -1.8032696
birminghamLongHigh = -1.618237

regionLatLow = birminghamLatLow
regionLatHigh = birminghamLatHigh
regionLongLow = birminghamLongLow
regionLongHigh = birminghamLongHigh
dumpFileName = 'birmingham'



numLatSteps = 2
numLongSteps = 2

latStep = (regionLatHigh - regionLatLow)/numLatSteps
longStep = (regionLongHigh - regionLongLow)/numLongSteps

# connect to wigle
wigle = Wigle("brian.macn", "hardeight")

counter = 0
countersDoneSoFar = 0

for latCount in range(0, numLatSteps):
    for longCount in range(0, numLongSteps):
        
        # search for all ssids within Ireland
        latRange = ((regionLatLow + (latStep*latCount), regionLatLow + (latStep*(latCount + 1))))
        longRange = ((regionLongLow + (longStep*longCount), regionLongLow + (longStep*(longCount + 1))))
            
        if(counter >= countersDoneSoFar):
            
            wigleInIreland = network.search(latrange1=latRange[0], latrange2=latRange[1], longrange1=longRange[0], longrange2=longRange[1], resultsPerPage=1000)
            
            #wigleInIreland = wigle.search(lat_range=latRange, 
            #                                  long_range=longRange, 
            #                                  max_results=10000)
            
            print(str(counter) + ': ' + str(len(wigleInIreland)))
            with open('wigleDump_' + dumpFileName + '_' + str(latRange[0]) + '_'  + str(latRange[1]) 
                      + '_' + str(longRange[0]) + "_"  + str(longRange[1]) 
                      + '.json', 'w') as outfile:
                json.dump(wigleInIreland, outfile)

            time.sleep(10)
                  
        counter += 1
        


Visit https://wigle.net/account to collect your API token
Then add your credentials to /Applications/anaconda/envs/Python2/lib/python2.7/site-packages/pygle/config.pyc


HTTPError: 401 Client Error: Unauthorized for url: https://api.wigle.net/api/v2/network/search?longrange1=-1.8032696&longrange2=-1.7107533&latrange1=52.3912092&resultsPerPage=1000&latrange2=52.4356646

In [7]:
from wigle import *
wigle = Wigle("brian.macn", "hardeight")
wigle.get_user_info()

WigleAuthenticationError: Could not authenticate as user brian.macn

In [8]:
len(wigleInIreland)

NameError: name 'wigleInIreland' is not defined

In [ ]:
import json
import glob

read_files = glob.glob("wigleDump*.json")
output_list = []

for f in read_files:
    with open(f, "rb") as infile:
        json_piece = json.load(infile)

        output_list = output_list + json_piece

print(len(output_list))

with open("merged_wigle_dumps_cork_talk.json", "wb") as outfile:
    json.dump(output_list, outfile)

Generate lat and long for each SSID in the devices list

In [ ]:
latLongList = dict()

# loop through the SSIDs
for ssid in ssidsFound:

    # If we haven't already looked for this SSID then do a wigle search for it
    if not ssid in latLongList:
        loc = findWigleSSID(ssid)

        # If wigle found a location store it
        if not (loc["lat"] == 0 and loc["long"] == 0):
            latLongList["ssid"] = {"lat":loc["lat"], "long":loc["long"]}


Read lats/longs from the wigle dump and average out lats/longs where ssid show up multiple times.

In [ ]:
import json
import string

# Import the Wigle dump
ssidList = json.loads(open('merged_wigle_dumps_derrycity.json').read())

# Start an empty dictionary
ssidLatLong = dict()

# Loop through each ssid json object and add it to the dictionary
for ssidObj in ssidList:

    # Extract the important details from the ssid json object
    ssidName = ssidObj["ssid"] 
    
    
    if(not ssidName == None and all(c in string.printable for c in ssidName) and (len(ssidName) > 3)): 

        ssidName = ssidName.replace("&amp;", "")
        ssidName = ssidName.replace("&", "")
        ssidName = ssidName.replace("<", "")
        ssidName = ssidName.replace(">", "")
        ssidLat = ssidObj["trilat"]
        ssidLong = ssidObj["trilong"]


        # if the ssid is not in the dictionary yet add it
        if not ssidName in ssidLatLong:

            ssidLatLong[ssidName] = {"lat":ssidLat, "long":ssidLong, "count":1}

        # if the ssid is already in the ditionary then average out this new entry
        else:

            ssidLatLong[ssidName]["lat"] = (ssidLatLong[ssidName]["lat"]*ssidLatLong[ssidName]["count"] + ssidLat)/(ssidLatLong[ssidName]["count"] + 1)
            ssidLatLong[ssidName]["long"] = (ssidLatLong[ssidName]["long"]*ssidLatLong[ssidName]["count"] + ssidLong)/(ssidLatLong[ssidName]["count"] + 1)
            ssidLatLong[ssidName]["count"] += 1


In [ ]:
for ssid in ssidLatLong:
    print ssid #+ str(ssidLatLong[ssid]["lat"]) + str(ssidLatLong[ssid]["long"])

Write out an XML file containing lat/longs of ssids spotted

In [ ]:
# Horrible manual writing of an XML file!!!!
with open('ssidsFound.xml', 'w') as xmlfile:
        
    xmlfile.write("<?xml version=\"1.0\"?>\n")
    xmlfile.write("<markers>\n")
    
    for ssid in ssidsFound:    
        if ssid in ssidLatLong:
            xmlfile.write("\t<marker>\n")
            xmlfile.write("\t\t<name>" + ssid + "</name>\n")
            xmlfile.write("\t\t<lat>" + str(ssidLatLong[ssid]["lat"]) + "</lat>\n")
            xmlfile.write("\t\t<lng>" + str(ssidLatLong[ssid]["long"]) + "</lng>\n")
            xmlfile.write("\t</marker>\n")

    xmlfile.write("</markers>\n")
        

Write an XML file with all SSIDS downloaded from wigle

In [ ]:
# Horrible manual writing of an XML file!!!!
with open('ssidsAll.xml', 'w') as xmlfile:
        
    xmlfile.write("<?xml version=\"1.0\"?>\n")
    xmlfile.write("<markers>\n")
    
    for ssid in ssidLatLong:
        #print(ssid)
        if not ssid == None:
            xmlfile.write("\t<marker>\n")
            xmlfile.write("\t\t<name>" + ssid + "</name>\n")
            xmlfile.write("\t\t<lat>" + str(ssidLatLong[ssid]["lat"]) + "</lat>\n")
            xmlfile.write("\t\t<lng>" + str(ssidLatLong[ssid]["long"]) + "</lng>\n")
            xmlfile.write("\t</marker>\n")

    xmlfile.write("</markers>\n")
        

# MacLocate Lookup Version

In [ ]:
from maclocate import *

locate("RDS-FREEWIFI")



In [ ]:
latLongList = []
for d in devices:
    ssids = devices[d]["ssids"]
    for ssid in ssids:
        locate(ssid)
        print loc
        latLongList.append(loc)
        

In [ ]:
latLongList

# Drawing Pictures 

In [ ]:
import mpld3

mpld3.enable_notebook()


# Another Dynamic Version

In [ ]:
#!/usr/bin/env python
# import all the needed libraries
import sys
from netaddr import *
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *
from subprocess import *
import datetime
import time

# clear the console
call(["clear"])                                           

# set date-time parameters                                                          
today = datetime.date.today()                 
d=today.strftime("%d, %b %Y")
tf=time.strftime(" %H:%M")
t=time.strftime(" %H:%M:%S")

# define variables                                                          
clients = []                          
uni = 0
mach = []
manu =[]

# our packet handler                                                          
def phandle(p):                       
    global uni    
    if p.haslayer(Dot11ProbeReq):                         
        mac = str(p.addr2)
        if p.haslayer(Dot11Elt):                          
            if p.ID == 0: 
                ssid = p.info                             
                if ssid not in clients and ssid != "":
                    clients.append(ssid)          
                    maco = EUI(mac)
            macf = maco.oui.registration().org   
            print (len(clients),mac+" ("+macf+") <--Probing--> "+ssid)
            if args.log:
                f.write (str(len(clients))+" "+mac+" ("+macf+") //"+" <--Probing--> "+ssid+"\n")
                if mac not in mach:
                            mach.append(mac)
                            uni+=1                                      
            elif mac not in mach:
                mach.append(mac)
                uni+=1                           

# our main function             
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description='PyRobe Help')
    parser.add_argument('interface', action="store", help="specify interface (ex. mon0)", default=False)
    parser.add_argument("-l","--log", dest="log",action="store_true", help="print log file")
    args = parser.parse_args()
    if args.log:
        f = open("ProbeLog"+str(today)+str(tf)+".txt","w")    
        sniff(iface=args.interface,prn=phandle, store=0)                    
        print ("\n")
        print ("Unique MACs: ",uni)
        f.write ("\nUnique MACs: "+str(uni))
        f.write ("\nScan performed on: "+str(d)+" at"+str(t))  
        f.close()                                                 
        print ("Log successfully written. Exiting!")
    else:
        sniff(iface=args.interface,prn=phandle, store=0)
        print ("\nSuccessfully Exited! No log file written.")

# Yet Another Dynamic Version
Works when called from a scripts, but nice becuase it does channel hopping

In [ ]:
import re
import pickle
from threading import Timer
import subprocess
from subprocess import call
import os.path
import sys

# config
database = 'allPairs.p'
channels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

if len(sys.argv) > 1:
    database = sys.argv[1]
if len(sys.argv) > 2:
    channels = sys.argv[2:len(sys.argv)]

# run
hopTime = 10

airport = '/System/Library/PrivateFrameworks/Apple80211.framework/Versions/Current/Resources/airport'

call(['networksetup',
    '-setairportpower',
    'airport', 'on'])

call(['sudo',
    airport,
    '--disassociate'])

channelIndex = int(0)
def hopChannels():
    global channelIndex
    channelIndex += 1
    if channelIndex == len(channels):
        channelIndex = 0
    channel = channels[channelIndex]
    print('hopping to channel ' + str(channel))
    call(['sudo',
        airport,
        '--channel=' + str(channel)])
    Timer(hopTime, hopChannels).start()
hopChannels()

allPairs = set();

if os.path.isfile(database):
    print('loading database from disk')
    allPairs = pickle.load(open(database, "rb"))

p = subprocess.Popen(('sudo',
    'tcpdump', '-l', '-e', '-I',
    'type mgt subtype probe-req'),
    stdout=subprocess.PIPE)
try:
    for row in p.stdout:
        result = row.decode('utf-8')
        pattern = re.compile('SA:(.+) \(oui Unknown\) Probe Request \((.+)\) ');
        m = pattern.search(result)
        if m:
            print(m.group(1) + ' @ ' + m.group(2))
            allPairs.add((m.group(1), m.group(2)))
            pickle.dump(allPairs, open(database, "wb"))
except KeyboardInterrupt:
    p.terminate()
    call(['networksetup',
        '-setairportpower',
        'airport', 'off'])